IMPORTANT

Before running the cells below, run in the terminal: `conda activate transformersum` and restart this notebook.  

In [1]:
import torch
import math
import syntok.segmenter as segmenter
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from itertools import zip_longest

# Overview

## Abstractive Summarization Models

At the time of this notebook, there are 3 state of the art abstractive text summarization models: 
1. Pegasus
2. T5
3. BART

Rather than comparing all 3 in this notebook, I have chosen to focus on BART.  BART has been shown to slightly outperform the others for summarization tasks (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7720861/).  

## BART Models

BART (https://arxiv.org/abs/1910.13461) was trained on a number of tasks.  There is no single BART model on HuggingFace for all of the training tasks presented in the paper, rather a bunch of models that are trained on the specific tasks.  For summarization, you can find BART models on HuggingFace that were trained on the following datasets:

* CNN/DailyMail - Dataset of news articles.  The summaries in this dataset resemble the source sentences and extractive summarization models perform well here.  BART beat all previous benchmarks. Use model ID = "facebook/bart-large-cnn"
* XSUM - Dataset of news articles.  The summaries in this dataset are highly abstract and generally much shorter than the original text.  Abstractive summarization models perform well here.  BART beat all previous benchmarks.  Use model ID = "facebook/bart-large-xsum"

Since there are 2 model choices, the one you should use depends on what you want the summarizer to do.  If it should be very high level and very short, go with XSUM.  If it should be more detailed but capture the basics of the parent text, then go with CNN.  

### Model Speed

Both models have distilled versions (smaller models for faster inference).  Since these models will be used in production environments, inference speed is important.  For this reason, I will evaluate the distilled versions of the models in this notebook.  Metrics showing how the distilled models compare to the full sized (baseline) models can be found here: https://huggingface.co/sshleifer/distilbart-xsum-12-6#metrics-for-distilbart-models.  

Dynamic quantization can be used to further reduce the model size and increase inference speed.  Pytorch makes this extremely simple with a 1-liner.  Quantization compresses the weights of a pre-trained model without much impact to model performance.  See an example with Pytorch here: https://pytorch.org/tutorials/intermediate/dynamic_quantization_bert_tutorial.html.

# Preparing the Models

In [2]:
# specify the model names for HuggingFace's model hub and the local paths to save/load them
bart_xsum = "sshleifer/distilbart-xsum-12-6"
bart_cnn = "sshleifer/distilbart-cnn-12-3"

bart_xsum_model_path = Path("models/bart_xsum")
bart_cnn_model_path = Path("models/bart_cnn")

In [3]:
# load the tokenizer and model from HuggingFace
bart_xsum_tokenizer = AutoTokenizer.from_pretrained(bart_xsum)
bart_xsum_model = AutoModelForSeq2SeqLM.from_pretrained(bart_xsum)

bart_cnn_tokenizer = AutoTokenizer.from_pretrained(bart_cnn)
bart_cnn_model = AutoModelForSeq2SeqLM.from_pretrained(bart_cnn)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/583M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/238 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/973M [00:00<?, ?B/s]

In [4]:
bart_xsum_model_path = Path("models/bart_xsum")
bart_cnn_model_path = Path("models/bart_cnn")

In [5]:
# quantize and save the compressed models
bart_xsum_model_quantized = torch.quantization.quantize_dynamic(
    bart_xsum_model, {torch.nn.Linear}, dtype=torch.qint8
)
torch.save(bart_xsum_model_quantized, bart_xsum_model_path / "model.pt")


bart_cnn_model_quantized = torch.quantization.quantize_dynamic(
    bart_cnn_model, {torch.nn.Linear}, dtype=torch.qint8
)
torch.save(bart_cnn_model_quantized, bart_cnn_model_path / "model.pt")

# Simple Summarization

In [6]:
articles = [
    ("Denmark has agreed to provide Ukraine with a Harpoon launcher and missiles to 'help Ukraine defend its coast',"
     "US Secretary of Defense Lloyd Austin said at the conclusion of the second Ukraine Contact Group meeting hosted"
     "by Austin on Monday. The second contact group meeting was held virtually. "
     "The Czech Republic also agreed to send 'substantial support' to Ukraine including 'a recent donation of attack "
     "helicopters, tanks and rocket systems,' Austin said at a press conference at the conclusion of the meeting."
     "Overall, 20 countries 'announced new security assistance packages,' after the meeting, Austin said, including "
     "'donating critically needed artillery ammunition, coastal defense systems and tanks and other armored vehicles.'"
     "'Others came forward with new commitments for training Ukraine’s forces and sustaining its military systems,' "
     "Austin added. "
     "A total of 47 countries participated in the contact group’s second meeting, Chairman of the Joint Chiefs of "
     "Staff Gen. Mark Milley said."
     "Secretary Austin will host the third meeting of the Ukraine Contact Group in person in Brussels on June 15, "
     "Austin said at the conclusion of the second virtual meeting of the contact group Monday. "
     "'I will convene the Contact Group for our third meeting next month and will gather in person this time, on "
     "June 15, in the margins of the NATO defense ministerial in Brussels,' Austin said. 'Of course, it won't be a "
     "NATO event, but we want to keep up the, up, keep up the tempo of these meetings and I wanted to use my travel "
     "to Europe to ensure that we're building on our momentum.'"
    ),
    ("Germany's vice chancellor and economy minister told CNN a recession is not inevitable. "
     "Speaking to CNN's Julia Chatterley at the World Economic Forum in Davos, Robert Habeck insisted that "
     "'nothing is inevitable, we are human beings and can change the course of history.'"
     "He also spoke to CNN about the war in Ukraine and Europe’s efforts to lessen dependence on Russian energy. "
     "Asked whether the European Union could reach an agreement on the next round of sanctions, including an oil "
     "embargo, he said he was confident a deal could be reached and could be done within days."
     "'I expect everyone — also Hungary — that they work to find a solution and not saying 'OK we have an exception "
     "and then we will lay back and build on our partnership with Putin,' he said while speaking earlier on a panel "
     "at Davos."
     "Habeck also discussed Germany’s dependence on Russian gas, saying German industry would collapse without "
     "Russian energy. Asked whether Germany would pay for Russian gas in rubles, Habeck said that German companies "
     "would pay for gas in euros, if Russia then decided to exchange those euros into rubles, it was a 'face saving' "
     "measure for Putin."
     "He insisted that any such moves were approved by the EU Commission and did not break sanctions."
     "More background: Russian President Vladimir Putin said in March that 'unfriendly' nations would have to pay "
     "rubles, rather than the euros or dollars stated in contracts. Buyers could make euro or dollar payments into "
     "an account at Russia's Gazprombank, which would then convert the funds into rubles and transfer them to a "
     "second account from which the payment to Russia would be made."
     "Gas supplies to Poland and Bulgaria were cut off, after they refused to pay in rubles. Other big European "
     "gas companies have told CNN they are working on ways to pay for Russian gas, while not breaking EU sanctions."
    ),
    ("Coffee giant Starbucks says it has exited Russia and will no longer have a brand presence there, according "
     "to a press release on Monday. "
     "The coffee company says it has been operating in Russia for 15 years and has now closed its 130 licensed "
     "cafes in the country. Starbucks joins other companies like McDonald’s and Exxon Mobil in taking its business "
     "completely out of Russia. "
     "Starbucks says it will 'support' its nearly 2,000 workers in Russia, including pay for six months and "
     "assistance for partners to transition to new opportunities outside of Starbucks. "
     "This comes after Starbucks CEO Kevin Johnson said in March that it had suspended all business activity in "
     "Russia, including shipment of all Starbucks products. "
    ),
]

In [7]:
summarizer_xsum = pipeline("summarization", model=bart_xsum_model_quantized, tokenizer=bart_xsum_tokenizer)
summarizer_cnn = pipeline("summarization", model=bart_cnn_model_quantized, tokenizer=bart_cnn_tokenizer)

for article in articles:
    print("----------\nNews Article Summary\n")
    print(f"XSUM Summary:\n{summarizer_xsum(article)}")
    print(f"CNN Summary:\n{summarizer_cnn(article)}")

----------
News Article Summary

XSUM Summary:
[{'summary_text': ' The first meeting of the Contact Contact Group in the European Union has been held in Brussels.'}]
CNN Summary:
[{'summary_text': " The Czech Republic also agreed to send 'substantial support' to Ukraine . 'We will always always be able to make a point point clear clear of the idea,' said Austin . 'I would have been a great part of the country's armed armed armed with missiles,' says Austin ."}]
----------
News Article Summary

XSUM Summary:
[{'summary_text': ' The Russian President, Vladimir Putin, has been speaking to the BBC.'}]
CNN Summary:
[{'summary_text': " Russian President Vladimir Vladimir Putin said in March that 'unfriendly' He said he would be able to pay for the energy of energy . He spoke to a panel of the World Economic Forum in Davos . Germany's vice chancellor and economy minister told CNN a recession is not inevitable ."}]
----------
News Article Summary



Your max_length is set to 142, but you input_length is only 136. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=68)


XSUM Summary:
[{'summary_text': ' Starbucks has announced that the company has decided to withdraw from Russia.'}]
CNN Summary:
[{'summary_text': ' The coffee company says it has been operating in Russia . Starbucks says it will not have a brand presence there . Starbucks will be no longer have the presence at the site . Starbucks has been a hot spot for the first time . The company says the company has a history history of working together .'}]


# Manipulating the Summary with Constrained Beam Search

Autoregressive models generate 1 word at a time, based on the word with the highest probability of occurring next.  The problem with a greedy search strategy like this is that it can be repetitive, and it could result in output that is suboptimal.  For example, if the next word is highly probable, but the word options after that (further down the search tree) are less probable, that could be a sub-optimal output.  See: https://huggingface.co/blog/how-to-generate.

Constrained beam searches inject desired words into the output sequence: https://huggingface.co/blog/constrained-beam-search.  Here I will try injecting some words into the summaries.

## Injecting Words into the Summary

The words we want inserted will come in a list.  The model will decide on the best place to put them.

In [8]:
force_words = [
    ["Denmark", "Ukraine", "Russia"],
    ["Germany", "European Union", "sanctions", "Russia"],
    ["Starbucks", "Russia", "employees"],
]
force_words = [[" " + str(w) + " " for w in wl] for wl in force_words]
print(force_words)

[[' Denmark ', ' Ukraine ', ' Russia '], [' Germany ', ' European Union ', ' sanctions ', ' Russia '], [' Starbucks ', ' Russia ', ' employees ']]


In [9]:
for article_id, article in enumerate(articles):
    print("----------\nNews Article Summary\n")
    
    input_ids = bart_xsum_tokenizer(article, return_tensors="pt").input_ids
    force_words_ids = bart_xsum_tokenizer(force_words[article_id], add_special_tokens=False).input_ids

    outputs = bart_xsum_model_quantized.generate(
        input_ids,
        force_words_ids=force_words_ids,
        num_beams=5,
        num_return_sequences=1,
        no_repeat_ngram_size=1,
        remove_invalid_values=True,
    )
    print(f"XSUM Summary:\n{bart_xsum_tokenizer.decode(outputs[0], skip_special_tokens=True)}")

    input_ids = bart_cnn_tokenizer(article, return_tensors="pt").input_ids
    force_words_ids = bart_cnn_tokenizer(force_words[article_id], add_special_tokens=False).input_ids

    outputs = bart_cnn_model_quantized.generate(
        input_ids,
        force_words_ids=force_words_ids,
        num_beams=5,
        num_return_sequences=1,
        no_repeat_ngram_size=1,
        remove_invalid_values=True,
    )
    print(f"CNN Summary:\n{bart_cnn_tokenizer.decode(outputs[0], skip_special_tokens=True)}")


----------
News Article Summary

XSUM Summary:
The Ukraine Contact Group is a Russia  in Ukraine  Denmark, and the Russian military has been involved on its own - but it's not yet to be an official or even fully funded for their defence. (as they are) 'the first time.'' at this meeting...'. "I Denmark 
CNN Summary:
 The Ukraine Contact Group has been Russia-Ukraine Denmark  since 2007. 'I'm not a military or safety for the first time' says Austin, Texas and other officials from Kiev. "We will be able to get back together" A total of 47 countries participated in contact with troops on Ukrainian border country's borders (with Moscow)). Officials say they have signed off at least 20% more than 100 million people during their visit abroad where it is now known as Washington DC – - I would like us! We are also interested about bringing up our own weapons arsenal against this project without an individual person who sent them into danger’s ‘dangerous zone,' said one Ukraine  Russia 
--------

## Including Start Words

If you know that you want the summary to begin a certain way, you could enforce start words.  More abstractly, you could define a template that you want the summary to follow.  Unfortunately, that is not possible at this time.

I looked into adding start words.  It seems that this code (https://github.com/huggingface/transformers/blob/518bd02c9b71291333ef374f055a4d1ac3042654/src/transformers/generation_beam_search.py#L389) could be edited to enforce every beam to begin with the tokens you specify, and that would ensure that the beam search results in a summary that begins with your words.  However, it would take me a long time to implement in HuggingFace's source code.  Also, it would be difficult to ensure that only the first sentence in  the summary started with those words.  To implement start words, you would pretty much have to implement the entire template feature mentioned in their blog post.

## Returning Multiple Beams

Below I will experiment with the number of repeating n_grams and return multiple beams to see how they differ.

In [10]:
for article_id, article in enumerate(articles):
    print("----------\nNews Article Summary\n")
    
    input_ids = bart_xsum_tokenizer(article, return_tensors="pt").input_ids
    force_words_ids = bart_xsum_tokenizer(force_words[article_id], add_special_tokens=False).input_ids

    outputs = bart_xsum_model_quantized.generate(
        input_ids,
        num_beams=5,
        num_return_sequences=3,
        no_repeat_ngram_size=3,
        remove_invalid_values=True,
    )
    print("XSUM Summaries:\n")
    for i in outputs:
        print(bart_xsum_tokenizer.decode(i, skip_special_tokens=True), "\n")

    input_ids = bart_cnn_tokenizer(article, return_tensors="pt").input_ids
    force_words_ids = bart_cnn_tokenizer(force_words[article_id], add_special_tokens=False).input_ids

    outputs = bart_cnn_model_quantized.generate(
        input_ids,
        num_beams=5,
        num_return_sequences=3,
        no_repeat_ngram_size=3,
        remove_invalid_values=True,
    )
    print("CNN Summaries: \n")
    for i in outputs:
        print(bart_cnn_tokenizer.decode(i, skip_special_tokens=True), "\n")


----------
News Article Summary

XSUM Summaries:

The first meeting of the Ukraine Contact Group has been held in Brussels. 

The first meeting of the Ukraine Contact Group has been held in the UK. 

The first meeting of the Ukraine Contact Group has been held in the Czech Republic. 

CNN Summaries: 

 U.S. Secretary of Defense said at the conclusion of the second meeting of the Ukraine Contact Group. 'I will convene the Contact Group for our third meeting,' said Austin. 'We will be able to keep up the tempo of these meetings,' says Austin, Texas. 

 U.S. Secretary of Defense said at the conclusion of the second meeting of the Ukraine Contact Group. 'I will convene the Contact Group for our third meeting,' said Austin. 'We will be able to keep up the tempo of these meetings,' says Austin. 'I would have to do it' 

 U.S. Secretary of Defense said at the conclusion of the second meeting of the Ukraine Contact Group. 'I will convene the Contact Group for our third meeting,' said Austin. '

# Summarizing by Paragraph

I wondered how the summaries would change if they were given fewer sentences.  Here I will segment the documents by sentence, group the sentences by a certain number to simulate paragraphs, and summarize the paragraphs.

In [11]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [12]:
for article_id, article in enumerate(articles):
    sents = [sent for paragraph in segmenter.analyze(article) for sent in paragraph]
    sents = [(''.join(str(t) for t in s).strip()) for s in sents]
    print("----------\nNews Article Summary\n")
    
    pg = 1
    sents_per_paragraph = 3
    for grouped in grouper(sents, sents_per_paragraph):
        grouped_sents = " ".join([g for g in grouped if g is not None])

        print(f"----------\nNews Article Paragraph {pg} of {math.ceil(len(sents) / sents_per_paragraph)}\n")
        pg += 1

        input_ids = bart_xsum_tokenizer(grouped_sents, return_tensors="pt").input_ids

        outputs = bart_xsum_model_quantized.generate(
            input_ids,
            num_beams=5,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
        )
        print("XSUM Summaries:\n")
        for i in outputs:
            print(bart_xsum_tokenizer.decode(i, skip_special_tokens=True), "\n")

        input_ids = bart_cnn_tokenizer(grouped_sents, return_tensors="pt").input_ids

        outputs = bart_cnn_model_quantized.generate(
            input_ids,
            num_beams=5,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
        )
        print("CNN Summaries: \n")
        for i in outputs:
            print(bart_cnn_tokenizer.decode(i, skip_special_tokens=True), "\n")


----------
News Article Summary

----------
News Article Paragraph 1 of 3

XSUM Summaries:

 The BBC has been following a series of news stories from the UK and other countries in Ukraine, which will be followed by an emergency meeting on Monday.The first time they have met with their own members to discuss what is being put up for this year's fight against Russia - its European Union (EU 

CNN Summaries: 

 Denmark has agreed to send'substantial' support for Ukraine. The second contact group meeting was held by the Czech Republic, who also sent a report on Russia's national security. It is not clear that it will be used in any country where or at least one of these countries are known as an anti-Russian attack site last year and more than 100 million people have spent their lives fighting against each other so they can get into danger from being hit with missiles like this would make him feel safe if you don’t do your own name? Send us all good news live online! Check out here: http:/

# Conclusions

Constrained beam search tends to make the summaries worse.  They read better when the model is allowed to generate the summary without influence.  

Summaries are not guaranteed to have correct information.  Sometimes they make no sense.  Other times they directly contradict the original text.

The summaries seem to get worse with fewer sentences, partly because the model starts predicting words from outside the domain of the article.  Paragraph level summarization is likely not feasible at this time.  

## What HuggingFace Has to Say

Currently HuggingFace has no extractive summarization models.  There are open feature requests for them, and the consensus is that abstractive summarizers are too risky to use in production, because they can produce sentences that do not make sense or are blatently false.  

https://github.com/huggingface/transformers/issues/4332

https://github.com/huggingface/transformers/issues/12460

----------------

# Overview

## Extractive Summarization Models

At the time of this notebook, there are several state of the art extractive summarization models.  They are evaluated using Rouge scores: R1, R2, RL.  The current leader is MatchSum, and its code is available here: https://github.com/maszhongming/MatchSum.  MatchSum does not have a ready-to-run form though, so there are 2 other models shown for comparison that do have ready-to-run forms, where they can simply be downloaded and called in a library.  

| Model | R1 | R2 | RL | Model Size (Mb) |
| --- | --- | --- | --- | --- |
| MatchSum (RoBERTa-base) | 44.41 | 20.86 | 40.55 | 816 |
| Distil-RoBERTa | 42.87 | 20.02 | 39.31 | 935 |
| Mobile BERT | 42.01 | 19.31 | 38.53 | 282 |

There are larger versions of RoBERTA and BERT, but the distilled and mobile versions are smaller, faster, and have comparable Rouge scores.  So I have chosen to focus on Mobile BERT in this notebook.  Their Rouge scores came from: https://transformersum.readthedocs.io/en/latest/extractive/models-results.html.  

Rouge score explanation: https://www.freecodecamp.org/news/what-is-rouge-and-how-it-works-for-evaluation-of-summaries-e059fb8ac840/

## Mobile BERT

Mobile BERT comes from the TransformerSum package, which can be found here: https://github.com/HHousen/TransformerSum.  It is designed to run like a HuggingFace model.  It was trained on the following datasets:

* CNN/DailyMail - Dataset of news articles.  The summaries in this dataset resemble the source sentences and extractive summarization models perform well here. 

### Model Speed

Whereas distil BERT compresses by reducing model depth, mobile BERT compresses by reducing model width.  Both methods result in faster inference.  Since these models will be used in production environments, inference speed is important. 

Dynamic quantization can be used to further reduce the model size and increase inference speed.  Pytorch makes this extremely simple with a 1-liner.  Quantization compresses the weights of a pre-trained model without much impact to model performance.  See an example with Pytorch here: https://pytorch.org/tutorials/intermediate/dynamic_quantization_bert_tutorial.html.

# Preparing the Model

I pulled the transformersum package's src folder from: https://github.com/HHousen/TransformerSum/tree/master/src, copied it into the directory with this notebook, and renamed it to transformersum, because there is no way to easily install it with pip.  Then I downloaded the model checkpoint from https://transformersum.readthedocs.io/en/latest/extractive/models-results.html#pretrained-ext and saved it to the models/mobile_bert_extr folder.  

In [13]:
from transformersum.extractive import ExtractiveSummarizer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
model = ExtractiveSummarizer.load_from_checkpoint("models/mobile_bert_extr/epoch=3.ckpt", strict=False)

/Users/nicholaslincoln/opt/anaconda3/envs/transformersum/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:213: UserWarning: Found keys that are in the model state dict but not in the checkpoint: ['word_embedding_model.embeddings.position_ids']
  rank_zero_warn(


In [15]:
articles = [
    ("Denmark has agreed to provide Ukraine with a Harpoon launcher and missiles to 'help Ukraine defend its coast',"
     "US Secretary of Defense Lloyd Austin said at the conclusion of the second Ukraine Contact Group meeting hosted"
     "by Austin on Monday. The second contact group meeting was held virtually. "
     "The Czech Republic also agreed to send 'substantial support' to Ukraine including 'a recent donation of attack "
     "helicopters, tanks and rocket systems,' Austin said at a press conference at the conclusion of the meeting."
     "Overall, 20 countries 'announced new security assistance packages,' after the meeting, Austin said, including "
     "'donating critically needed artillery ammunition, coastal defense systems and tanks and other armored vehicles.'"
     "'Others came forward with new commitments for training Ukraine’s forces and sustaining its military systems,' "
     "Austin added. "
     "A total of 47 countries participated in the contact group’s second meeting, Chairman of the Joint Chiefs of "
     "Staff Gen. Mark Milley said."
     "Secretary Austin will host the third meeting of the Ukraine Contact Group in person in Brussels on June 15, "
     "Austin said at the conclusion of the second virtual meeting of the contact group Monday. "
     "'I will convene the Contact Group for our third meeting next month and will gather in person this time, on "
     "June 15, in the margins of the NATO defense ministerial in Brussels,' Austin said. 'Of course, it won't be a "
     "NATO event, but we want to keep up the, up, keep up the tempo of these meetings and I wanted to use my travel "
     "to Europe to ensure that we're building on our momentum.'"
    ),
    ("Germany's vice chancellor and economy minister told CNN a recession is not inevitable. "
     "Speaking to CNN's Julia Chatterley at the World Economic Forum in Davos, Robert Habeck insisted that "
     "'nothing is inevitable, we are human beings and can change the course of history.'"
     "He also spoke to CNN about the war in Ukraine and Europe’s efforts to lessen dependence on Russian energy. "
     "Asked whether the European Union could reach an agreement on the next round of sanctions, including an oil "
     "embargo, he said he was confident a deal could be reached and could be done within days."
     "'I expect everyone — also Hungary — that they work to find a solution and not saying 'OK we have an exception "
     "and then we will lay back and build on our partnership with Putin,' he said while speaking earlier on a panel "
     "at Davos."
     "Habeck also discussed Germany’s dependence on Russian gas, saying German industry would collapse without "
     "Russian energy. Asked whether Germany would pay for Russian gas in rubles, Habeck said that German companies "
     "would pay for gas in euros, if Russia then decided to exchange those euros into rubles, it was a 'face saving' "
     "measure for Putin."
     "He insisted that any such moves were approved by the EU Commission and did not break sanctions."
     "More background: Russian President Vladimir Putin said in March that 'unfriendly' nations would have to pay "
     "rubles, rather than the euros or dollars stated in contracts. Buyers could make euro or dollar payments into "
     "an account at Russia's Gazprombank, which would then convert the funds into rubles and transfer them to a "
     "second account from which the payment to Russia would be made."
     "Gas supplies to Poland and Bulgaria were cut off, after they refused to pay in rubles. Other big European "
     "gas companies have told CNN they are working on ways to pay for Russian gas, while not breaking EU sanctions."
    ),
    ("Coffee giant Starbucks says it has exited Russia and will no longer have a brand presence there, according "
     "to a press release on Monday. "
     "The coffee company says it has been operating in Russia for 15 years and has now closed its 130 licensed "
     "cafes in the country. Starbucks joins other companies like McDonald’s and Exxon Mobil in taking its business "
     "completely out of Russia. "
     "Starbucks says it will 'support' its nearly 2,000 workers in Russia, including pay for six months and "
     "assistance for partners to transition to new opportunities outside of Starbucks. "
     "This comes after Starbucks CEO Kevin Johnson said in March that it had suspended all business activity in "
     "Russia, including shipment of all Starbucks products. "
    ),
]

In [16]:
for article in articles:
    print("----------\nNews Article Summary\n")
    print(f"Mobile BERT Extractive Summary:\n{model.predict(article, num_summary_sentences=3)}")

----------
News Article Summary

Mobile BERT Extractive Summary:
The Czech Republic also agreed to send ' substantial support ' to Ukraine including ' a recent donation of attack helicopters , tanks and rocket systems , ' Austin said at a press conference at the conclusion of the meeting. A total of 47 countries participated in the contact group ’s second meeting , Chairman of the Joint Chiefs of Staff Gen. Mark Milley said. Secretary Austin will host the third meeting of the Ukraine Contact Group in person in Brussels on June 15 , Austin said at the conclusion of the second virtual meeting of the contact group Monday '.
----------
News Article Summary

Mobile BERT Extractive Summary:
Speaking to CNN 's Julia Chatterley at the World Economic Forum in Davos , Robert Habeck insisted that ' nothing is inevitable , we are human beings and can change the course of history. 'He also spoke to CNN about the war in Ukraine and Europe ’s efforts to lessen dependence on Russian energy. Habeck als

In [17]:
for article in articles:
    print("----------\nNews Article Summary\n")
    print(f"Mobile BERT Extractive Summary:\n{model.predict(article, num_summary_sentences=2)}")

----------
News Article Summary

Mobile BERT Extractive Summary:
The Czech Republic also agreed to send ' substantial support ' to Ukraine including ' a recent donation of attack helicopters , tanks and rocket systems , ' Austin said at a press conference at the conclusion of the meeting. Secretary Austin will host the third meeting of the Ukraine Contact Group in person in Brussels on June 15 , Austin said at the conclusion of the second virtual meeting of the contact group Monday '.
----------
News Article Summary

Mobile BERT Extractive Summary:
Speaking to CNN 's Julia Chatterley at the World Economic Forum in Davos , Robert Habeck insisted that ' nothing is inevitable , we are human beings and can change the course of history. 'He also spoke to CNN about the war in Ukraine and Europe ’s efforts to lessen dependence on Russian energy.
----------
News Article Summary

Mobile BERT Extractive Summary:
The coffee company says it has been operating in Russia for 15 years and has now clo

# Summarizing by Paragraph

I wondered how the summaries would change if they were given fewer sentences.  Here I will segment the documents by sentence, group the sentences by a certain number to simulate paragraphs, and summarize the paragraphs.

In [18]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [19]:
for article_id, article in enumerate(articles):
    sents = [sent for paragraph in segmenter.analyze(article) for sent in paragraph]
    sents = [(''.join(str(t) for t in s).strip()) for s in sents]
    print("----------\nNews Article Summary\n")
    
    pg = 1
    sents_per_paragraph = 4
    for grouped in grouper(sents, sents_per_paragraph):
        grouped_sents = " ".join([g for g in grouped if g is not None])

        print(f"----------\nNews Article Paragraph {pg} of {math.ceil(len(sents) / sents_per_paragraph)}\n")
        pg += 1

        print(f"Mobile BERT Extractive Summary:\n{model.predict(grouped_sents, num_summary_sentences=2)}")


----------
News Article Summary

----------
News Article Paragraph 1 of 2

Mobile BERT Extractive Summary:
The second contact group meeting was held virtually. The Czech Republic also agreed to send ' substantial support ' to Ukraine including ' a recent donation of attack helicopters , tanks and rocket systems , ' Austin said at a press conference at the conclusion of the meeting.
----------
News Article Paragraph 2 of 2

Mobile BERT Extractive Summary:
Secretary Austin will host the third meeting of the Ukraine Contact Group in person in Brussels on June 15 , Austin said at the conclusion of the second virtual meeting of the contact group Monday '. I will convene the Contact Group for our third meeting next month and will gather in person this time , on June 15 , in the margins of the NATO defense ministerial in Brussels , ' Austin said '.
----------
News Article Summary

----------
News Article Paragraph 1 of 3

Mobile BERT Extractive Summary:
Speaking to CNN 's Julia Chatterley at 

# Conclusions

The extractive summaries are both easy to read and factually correct, even though they may not be as succinct as the abstractive summarizers.

Paragraph level summarization is possible and the results are good.  This shows promise.

Extractive summarization seems better fit for production applications.